In [ ]:
import pickle
import pandas as pd
# load pickled data
with open('./data/t15_copyTask.pkl', 'rb') as f:
    dat = pickle.load(f)
    
if isinstance(dat, dict):
    print("Keys in checkpoint:", dat.keys())
else:
    print("Loaded object type:", type(dat))

# dat["decoded_sentence"] is already a LIST of sentences
preds = dat["decoded_sentence"]

# BLEU requires list[list[str]]
refs = [[r] for r in dat["cue_sentence"]] # wrap each reference sentence in its own list

print(preds)
print(refs)


b2txt_csv_df = pd.read_csv("data/t15_copyTaskData_description.csv")
print(b2txt_csv_df)

Keys in checkpoint: dict_keys(['post_implant_day', 'vocab_size', 'cue_sentence', 'cue_sentence_phonemes', 'decoded_logits', 'decoded_phonemes_raw', 'decoded_sentence', 'decoded_sentence_phonemes', 'speech_duration_s'])
['clean that up', 'you feel bad', 'what do i have', 'i like music', 'you feel good', 'not very bad', 'they tell you i am comfortable', 'no they are not going', 'yes they have', 'you clean my computer', 'do you like that', 'you are right here', 'tell my family they are not comfortable', 'where is they help', 'my nurse is very good', 'i need music', 'what do you tell my nurse', 'hope you feel', 'goodbye what', 'what is okay', 'do i', 'yes i am tired', 'it is that bad', 'you are closer', 'are my glasses very good', 'tell my family i am not comfortable', 'is my family good', 'please have faith', 'my family is not going outside', 'no i do not feel very comfortable', 'you hope it is goodbye', 'you need it here', 'i like how comfortable my family is', 'very good here', 'tell my

In [7]:
import pickle
import evaluate
import numpy as np

# WER calculation function
def calculate_aggregate_error_rate(references, hypotheses):
    """
    Calculate Word Error Rate (WER) using edit distance.
    
    Args:
        references: list of reference sentences (ground truth)
        hypotheses: list of hypothesis sentences (predictions)
    
    Returns:
        WER as a float (0.0 to 1.0+)
    """
    def levenshtein_distance(ref, hyp):
        """Calculate edit distance between two sequences."""
        ref_len = len(ref)
        hyp_len = len(hyp)
        
        # Create distance matrix
        d = np.zeros((ref_len + 1, hyp_len + 1), dtype=int)
        
        # Initialize first row and column
        for i in range(ref_len + 1):
            d[i][0] = i
        for j in range(hyp_len + 1):
            d[0][j] = j
        
        # Calculate distances
        for i in range(1, ref_len + 1):
            for j in range(1, hyp_len + 1):
                if ref[i-1] == hyp[j-1]:
                    cost = 0
                else:
                    cost = 1
                
                d[i][j] = min(
                    d[i-1][j] + 1,      # deletion
                    d[i][j-1] + 1,      # insertion
                    d[i-1][j-1] + cost  # substitution
                )
        
        return d[ref_len][hyp_len]
    
    total_distance = 0
    total_ref_words = 0
    
    for ref, hyp in zip(references, hypotheses):
        ref_words = ref.split()
        hyp_words = hyp.split()
        
        distance = levenshtein_distance(ref_words, hyp_words)
        total_distance += distance
        total_ref_words += len(ref_words)
    
    if total_ref_words == 0:
        return 0.0
    
    wer = total_distance / total_ref_words
    return wer


bleu = evaluate.load("bleu")

# # Load pkl file
# with open('./data/t15_copyTask.pkl', 'rb') as f:
#     dat = pickle.load(f)

# dat["decoded_sentence"] is already a LIST of sentences
preds = dat["decoded_sentence"]                 # list[str]

# BLEU requires list[list[str]]
refs = [[r] for r in dat["cue_sentence"]]       # wrap each reference sentence in its own list

# Compute BLEU
bleu_result = bleu.compute(predictions=preds, references=refs)

print(f"BLEU score: {bleu_result['bleu']:.4f}")

# Compute WER (add this after your BLEU calculation)
refs_flat = dat["cue_sentence"]  # WER expects flat list[str]
wer_score = calculate_aggregate_error_rate(refs_flat, preds)
print(f"WER score: {wer_score*100:.2f}%")


BLEU score: 0.9179
WER score: 4.34%
